In [2]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, RepeatVector, TimeDistributed
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [3]:
class CharacterTable(object):
    def __init__(self, chars):        
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):        
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):        
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [4]:
TRAINING_SIZE = 50000
DIGITS = 3
MAXOUTPUTLEN = DIGITS + 1
MAXLEN = DIGITS + 1 + DIGITS

chars = '0123456789+ '
ctable = CharacterTable(chars)

In [5]:
def return_random_digit():
  return np.random.choice(list('0123456789'))  
  
def generate_number():
  num_digits = np.random.randint(1, DIGITS + 1)  
  return int(''.join( return_random_digit()
                      for i in range(num_digits)))

def data_generate(num_examples):
  questions = []
  expected = []
  seen = set()
  print('Generating data...')
  while len(questions) < TRAINING_SIZE:      
      a, b = generate_number(), generate_number()  
      #Remove already seen elements
      key = tuple(sorted((a, b)))
      if key in seen:
          continue
      seen.add(key)
      # Pad the data with spaces such that it is always MAXLEN.
      q = '{}+{}'.format(a, b)
      query = q + ' ' * (MAXLEN - len(q))
      ans = str(a + b)
      # Answers can be of maximum size DIGITS + 1.
      ans += ' ' * (DIGITS + 1 - len(ans))
      questions.append(query)
      expected.append(ans)
  print('Total addition questions:', len(questions))
  return questions, expected


def encode_examples(questions,answers):
  x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
  y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
  for i, sentence in enumerate(questions):
      x[i] = ctable.encode(sentence, MAXLEN)
  for i, sentence in enumerate(answers):
      y[i] = ctable.encode(sentence, DIGITS + 1)

  indices = np.arange(len(y))
  np.random.shuffle(indices)
  return x[indices],y[indices]

In [6]:
q,a = data_generate(TRAINING_SIZE)
x,y = encode_examples(q,a)
split_at = len(x) - len(x) // 10
x_train, x_val, y_train, y_val = x[:split_at], x[split_at:],y[:split_at],y[split_at:]


print('Training Data shape:')
print('X : ', x_train.shape)
print('Y : ', y_train.shape)

print('Sample Question(in decoded form) : ', ctable.decode(x_train[0]),'Sample Output : ', ctable.decode(y_train[0]))

Generating data...
Total addition questions: 50000
Training Data shape:
X :  (45000, 7, 12)
Y :  (45000, 4, 12)
Sample Question(in decoded form) :  18+140  Sample Output :  158 


In [7]:
model = Sequential()
#converts from 1*32 to 1 * 6
model.add(Dense(6, input_dim=10))
print(model.output_shape)
#converts from 1*6 to 1*3*6
model.add(RepeatVector(3))
print(model.output_shape) 
input_array = np.random.randint(1000, size=(1, 10))
print("Shape of input : ", input_array.shape)
model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
print("Shape of output : ", output_array.shape)
# note: `None` is the batch dimension
print('Input : ', input_array[0])
print('Output : ', output_array[0])

(None, 6)
(None, 3, 6)
Shape of input :  (1, 10)
Shape of output :  (1, 3, 6)
Input :  [555 206 196 315 542 445 831 861 838  81]
Output :  [[  12.334458  354.43985   760.5043   -576.0926   -408.6205   1022.52686 ]
 [  12.334458  354.43985   760.5043   -576.0926   -408.6205   1022.52686 ]
 [  12.334458  354.43985   760.5043   -576.0926   -408.6205   1022.52686 ]]


In [12]:
#Hyperaparams
RNN = layers.SimpleRNN
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
#ENCODING
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(RepeatVector(MAXOUTPUTLEN))
#DECODING
for _ in range(LAYERS):    
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               18048     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 4, 128)            32896     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 12)             1548      
Total params: 52,492
Trainable params: 52,492
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 5):
    print()  
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=20,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    print('Finished iteration ', iteration)
    numcorrect = 0
    numtotal = 20
    
    for i in range(numtotal):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Question', q, end=' ')
        print('True', correct, end=' ')
        print('Guess', guess, end=' ')
        if guess == correct :
          print('Good job')
          numcorrect += 1
        else:
          print('Fail')
         
    print('The model scored ', numcorrect*100/numtotal,' % in its test.')
        


Train on 45000 samples, validate on 5000 samples
Epoch 1/20
45000/45000 [==============================] - 3s 71us/step - loss: 1.6711 - acc: 0.3911 - val_loss: 1.5362 - val_acc: 0.4377
Epoch 2/20
45000/45000 [==============================] - 3s 74us/step - loss: 1.4392 - acc: 0.4677 - val_loss: 1.3404 - val_acc: 0.5006
Epoch 3/20
45000/45000 [==============================] - 3s 74us/step - loss: 1.2382 - acc: 0.5363 - val_loss: 1.1439 - val_acc: 0.5653
Epoch 4/20
45000/45000 [==============================] - 3s 73us/step - loss: 1.0408 - acc: 0.6056 - val_loss: 0.9668 - val_acc: 0.6302
Epoch 5/20
45000/45000 [==============================] - 3s 73us/step - loss: 0.8598 - acc: 0.6708 - val_loss: 0.8127 - val_acc: 0.6817
Epoch 6/20
45000/45000 [==============================] - 3s 73us/step - loss: 0.7079 - acc: 0.7310 - val_loss: 0.6780 - val_acc: 0.7407
Epoch 7/20
45000/45000 [==============================] - 3s 74us/step - loss: 0.5997 - acc: 0.7769 - val_loss: 0.5878 - val_acc